In [1]:
#from config import setup
#setup()

In [2]:
#!pip uninstall config

In [3]:
import numpy as np
from docplex.cp.model import CpoModel
from docplex.cp.parameters import CpoParameters

In [4]:
#!git clone https://github.com/tamy0612/JSPLIB/tree/master/instances
    
import os
f_list = os.listdir('instances/')
file_list =[]
for f in f_list:
    f = "instances/"+f
    file_list.append(f)
    
#print(file_list)

In [5]:
def cplex_makespan_model(n: int, m: int, durations, machines) -> CpoModel:
    # n number of jobs, m machines
    # 1 line for un job with tasks on m machines
       
    naive = np.sum(durations)
    
    mdl = CpoModel()
    
    #Une variable est l'intervalle de durée pour une tache 
    x = [[mdl.interval_var(size=durations[i,j], name="O{}-{}".format(i, j)) for j in range(m)] for i in range(n)]
    
    #contrainte end max d'une tache calculée avant
    for i in range(n): 
        for j in range(m):
            mdl.add(mdl.end_of(x[i][j]) <= naive)
            
    #precedence
    for i in range(n): #for each job
        for j in range(m-1): #taches ordonnées
            mdl.add(mdl.end_before_start(x[i][j], x[i][j+1]))
     
    # une machine ne peut faire qu'une tache à la fois
    listtaches = [[] for k in range(m)]
    for i in range(n):
        for j in range(m):
            listtaches[machines[i,j]].append(x[i][j])

    for k in range(m):
        mdl.add(mdl.no_overlap(listtaches[k]))

    
    makespan = mdl.integer_var(0,naive) 
    # le makespan est le max des tps pour chaque job
    mdl.add( makespan == mdl.max([mdl.end_of(x[i][m-1]) for i in range(n)]))
    
    mdl.add(mdl.minimize(makespan))
    
    return mdl, x

In [6]:
def cplex_solve_instance2 (filename, start,searchtype, interference, timelimit):

    f = open(filename, "r")
    content = f.read()
    f.close()
    #print(content)
    lines = content.split("\n")

    array = []

    for i in lines[start:]:
        numbers = i.split(" ")
        while numbers.count('') >0:
            numbers.remove('')
        for j in range(len(numbers)):
            numbers[j] = int(numbers[j])
        #print(numbers)
        if numbers != []:
            array.append(numbers)

    n = array[0][0] #number of jobs
    m = array[0][1] #number of machines

    machines = np.matrix(array[1:])[:,::2]
    
    durations = np.matrix(array[1:])[:,1::2]


    mdl, x = cplex_makespan_model(n, m, durations, machines)
    
    sol = mdl.start_search(TimeLimit=timelimit, SearchType= searchtype, DefaultInferenceLevel=interference).solve()
    #sol.print_solution()
    #display_sol (sol)
    
    return sol



In [7]:
#extrait du TP de PPC, permet de regarder les perfs avec CPLEX (à comparer avec métaheuristique après)

searchtype = "Restart"
interference1 = "Low"
interference2 = "Extended"
params = CpoParameters(TimeLimit=10, LogPeriod=100000)


sol1=[]
nodes1=[]
runtime1=[]

sol2=[]
nodes2=[]
runtime2=[]


times = [0.2,0.5,0.8,1.1,1.5,2,2.5,3,4,5,6,7,8,10]
nb1=[]
nb2=[]

nb11=0
nb22=0

done1 = []
done2 =[]

#file_list = file_list[0:5] #for quick tests

for t in times:
    #on va tester la resolutin avec des timetimits différents
    for filename in file_list[:5]: #:5 for tests
        #print(filename)
        
        if filename not in done1:
            s1 = cplex_solve_instance2 (filename, 4, searchtype, interference1, t) # we start at line 4 to parse due to the instance format
            sol1.append(s1)
            nodes1.append(sol1[-1].get_solver_infos()["NumberOfChoicePoints"])
            runtime1.append(sol1[-1].get_solver_infos()["TotalTime"])
            if sol1[-1].get_solver_infos()['SearchStatus']=='SearchCompleted':
                nb11+=1
                done1.append(filename)
        
        if filename not in done2:
            s2 = cplex_solve_instance2 (filename, 4, searchtype, interference2, t) # we start at line 4 to parse due to the instance format
            sol2.append(s2)
            nodes2.append(sol2[-1].get_solver_infos()["NumberOfChoicePoints"])
            runtime2.append(sol2[-1].get_solver_infos()["TotalTime"])
            if sol2[-1].get_solver_infos()['SearchStatus']=='SearchCompleted':
                nb22+=1
                done2.append(filename)
            
            
    nb1.append(nb11)
    nb2.append(nb22)

CpoSolverException: Solver error: Worker 0 raised exception: No more memory..

In [ ]:
import matplotlib.pyplot as plt2

plt2.plot(times, nb1, 'r', label = "interference Low")
plt2.plot(times, nb2, 'b', label = "interference Extended")
plt2.xlabel('time limit')
plt2.ylabel('number of instances solved')
plt2.legend()
plt2.show()

1 // Implémentation de la représentation détaillée d'une solution à un problème de Job Shop (appelée Schedule)

In [8]:
filename = file_list[0]
filename
#filename = 'aaa1'

'instances/abz5'

In [9]:
def generate_instance(filename, start):
    f = open(filename, "r")
    content = f.read()
    f.close()
    #print(content)
    lines = content.split("\n")

    array = []

    for i in lines[start:]:
        numbers = i.split(" ")
        while numbers.count('') >0:
            numbers.remove('')
        for j in range(len(numbers)):
            numbers[j] = int(numbers[j])
        #print(numbers)
        if numbers != []:
            array.append(numbers)

    n = array[0][0] #number of jobs
    m = array[0][1] #number of machines

    machines = np.matrix(array[1:])[:,::2]
    
    durations = np.matrix(array[1:])[:,1::2]

    return machines, durations, n, m

machines, durations, n, m = generate_instance(filename, 4) #we start at line 4 due to instance shape

In [10]:
print(machines)
print(n)
print(m)

[[4 8 6 5 1 2 9 7 0 3]
 [5 3 6 4 2 8 0 1 7 9]
 [9 8 0 1 6 5 7 4 2 3]
 [7 2 1 4 3 6 5 0 9 8]
 [3 4 9 8 0 2 6 5 7 1]
 [1 4 5 6 8 2 7 9 3 0]
 [7 1 4 3 0 8 2 5 6 9]
 [4 6 3 2 1 5 7 0 8 9]
 [0 6 3 7 1 2 4 5 8 9]
 [3 0 1 8 7 9 6 4 5 2]]
10
10


In [11]:
machines[2,0]

9

In [12]:
ex = np.random.permutation(10).tolist()
ex

[9, 4, 7, 8, 1, 5, 2, 6, 3, 0]

In [13]:
#init avec job representation
#permutation avec m valeures pour chaque job (n jobs)
def init_job(n,m):
    list_job = []
    for i in range(n):
        lis = [i] * m
        list_job.extend(lis)

    list_job = np.random.permutation(np.array(list_job))
    list_job.tolist()
    return list_job

In [14]:
# une solution de type ordre de passage sur les ressources represente:
# pour chaque machine l'odre dans lequel réaliser les diff op
# matrice de taille m (nbre machine) * n (nombres de job)
# sachant que chaque job contient exactement 1 operation par machine

#solution random: on genere une matrice m*n avec des nombres compris entre 0 et n-1 dns chaque ligne
#                sans duplicata et dans n'importe quel ordre au début

def init_sol_resources(n,m, machines):
    l = []
    for i in range(m):
        ligne = np.random.permutation(n).tolist()
        l.append(ligne)
    
    #print(l)
    
    #on a une liste de m listes (taille n)
    #on veut une liste de liste de tuple
    #pour chaque tuple:
    #  1er element numéro de job
    #  2eme element numéro de la machines dans les op du job
    
    tupl_l = []
    for i in range(m):
        tupl_list = []
        for j in range(n):
            job = l[i][j]
            ressource = i
            for k in range(m) :
                if machines[j,k] == i:
                    n_op = k
            tupl = (job, n_op)
            tupl_list.append(tupl)
        tupl_l.append(tupl_list)
    
    #print(tupl_l)
    
    return l, tupl_l

l, tupl_l = init_sol_resources(n, m, machines)

In [15]:
# une solution de type ordre de passage sur les ressources represente:
# pour chaque machine l'odre dans lequel réaliser les diff op
# matrice de taille m (nbre machine) * n (nombres de job)
# sachant que chaque job contient exactement 1 operation par machine

#solution random mais sans cycle car issu de job representation

def init_sol_resources_nocycle(n,m, machines):
    
    ressource = [[] for _ in range(m)] # matrice m*n
    
    list_job = init_job(n,m) #representation job aléatoire
    print(list_job)
    
    #on stocke à quelle tache on en est pour chacun des n job (max = m)
    state = [0]*n #au début 0 pour chaque mach
    
    for j in list_job:
        #trouver la machine utilisée pour le job j à la kème tache (k=state[j])
        mac = machines[j,state[j]]
        #on peut remplir ressource
        ressource[mac].append((j,state[j]))
        
        state[j]+=1      
    
    
    return list_job, ressource

list_job, ressource = init_sol_resources_nocycle(n, m, machines)

[1 5 8 9 9 4 9 8 3 0 9 4 2 9 7 2 2 9 6 8 2 6 8 5 1 3 7 2 2 3 1 0 6 5 7 0 0
 7 1 4 4 2 7 4 8 1 0 7 0 4 5 9 0 1 3 2 0 6 6 8 7 4 3 9 4 4 0 5 6 8 7 3 8 3
 6 0 3 1 5 1 3 8 5 5 2 9 8 3 1 6 9 6 7 6 5 5 4 1 7 2]


In [16]:
#ressource

In [33]:
# on va essayer de passer de la representation par ressource à la representation détaillée

# la representation détaillée: matrice de n lignes (nombre de job) et m colonnes (nbre machines)
# la jeme colonne correspond a la jieme op pour le job en question
# chaque case contient la date de début de l'op correspondante

import math
infini = math.inf



def ressource_to_detaillee(tupl_l, n, m, durations, machines):
       
    #on stocke à quelle tache en est chaque machine (max = n)
    state = [0 for k in range(m)] #au début 0 pour chaque machine
    
    #on créé la matrice qu'on essaie de créer, la representation détailléee
    detail = [[infini] * m for _ in range(n)] #np.zeros ((n*m)) #initialisation à 0
    
    it =  0
    
    #  "on recommence à chaque fois qu'on avance..""
    while (max(max(detail))>=infini): # and (iter<5000)
        
        move = 0 #on regarde si on a réussi à remplir au moins une case dans la dernière itération
        it+=1
        #print("restart")
        
        #on remplit colonne par colonne la représentation détaillée
        for j in range(m):
            for i in range(n):

                #on s'interesse à la tache (i,j) la jème tache pour le job i

                if detail[i][j] == infini: #sinon pas besoin de s'interesser à cette tache
                
                    #tps auquel la tache precedente pour le job sera finie
                    if j==0:
                        prec = 0
                    if j>0:
                        prec = detail[i][j-1] + durations[i,j-1]

                    #print("prec job: ", prec)

                    #on peut commencer seulement si machine necessaire pour la tache est libre
                    #machine necessaire -> regarder matrice machines

                    machine_used = machines[i,j]
                    st = state[machine_used] #quelle colonne de ressource representation est la suivante

                    job, num_op = tupl_l[machine_used][st]
   
                    if job != i:
                        #on ne peut pas faire la tache desuite, la machine a une autre tache de prévue
                        mac = infini
                    if job ==i:
                        if st ==0: #si st = 0, 1ère tache, la machine est prete
                            mac = 0
                        if st!=0:
                        #la machine fnit sa tache precedente puis c'est le tour de notre tache
                            prec_j, prec_tache_machine = tupl_l[machine_used][st-1]
                            mac = detail[prec_j] [prec_tache_machine] + durations[prec_j, prec_tache_machine]

                    #print("prec ressource: ", mac)
                    startdate = max(mac,prec)
                    detail[i][j] = startdate

                    #mise à jour de state
                    if startdate < infini:
                        state[machine_used]+=1
                        move += 1 #on a remplit une case
                        
        print("move after colonne : ", move)
        
                        
       
        
        if move ==0:
            print("no move")
            break

    return detail
            

    

In [34]:
detail = ressource_to_detaillee(ressource, n, m, durations, machines)
print(detail)

move after colonne :  14
move after colonne :  9
move after colonne :  19
move after colonne :  8
move after colonne :  17
move after colonne :  10
move after colonne :  8
move after colonne :  12
move after colonne :  1
[[0, 363, 644, 738, 837, 904, 993, 1070, 1169, 1255], [0, 246, 575, 694, 769, 863, 1426, 1518, 1600, inf], [0, 302, 363, 446, 511, 575, 660, 868, 1296, 1630], [0, 94, 597, 769, 868, 1155, 1284, 1350, 1426, 1489], [50, 119, 454, 536, 631, 993, 1060, 1155, 1254, 1600], [0, 305, 660, 738, 929, 1216, 1321, 1489, 1551, 1630], [358, 511, 597, 694, 790, 1009, 1150, 1409, 1508, 1648], [207, 305, 378, 460, 658, 837, 1169, 1255, 1556, inf], [0, 94, 165, 408, 729, 1060, 1150, 1226, 1284, 1551], [0, 94, 153, 235, 302, 358, 804, 953, 1350, 1409]]


In [35]:
list_job

array([1, 5, 8, 9, 9, 4, 9, 8, 3, 0, 9, 4, 2, 9, 7, 2, 2, 9, 6, 8, 2, 6,
       8, 5, 1, 3, 7, 2, 2, 3, 1, 0, 6, 5, 7, 0, 0, 7, 1, 4, 4, 2, 7, 4,
       8, 1, 0, 7, 0, 4, 5, 9, 0, 1, 3, 2, 0, 6, 6, 8, 7, 4, 3, 9, 4, 4,
       0, 5, 6, 8, 7, 3, 8, 3, 6, 0, 3, 1, 5, 1, 3, 8, 5, 5, 2, 9, 8, 3,
       1, 6, 9, 6, 7, 6, 5, 5, 4, 1, 7, 2])

In [30]:
def evaluate_detail(detail, n, m, machines, durations):
    fins = []
    for i in range(n):
        fin = detail[i][m-1]
        fins.append(fin)
    
    return max(fins)

evaluate_detail(detail, n, m, machines, durations)

1782

In [ ]:
def validate_detail():
    val = True
    for i in range(n):
        for j in range(1:m):    
            #precedence
            if detail[i][j]<detail[i][j-1]+durations[i][j-1]:
                print("not correct precedence for ligne ", i, 'colonne: ', j)
                val = False
                break
    
    
    #machine ne peut traiter qu'une tache à la fois
    for k in range(m):
        #on verifie qu'un machine ne fait qu'une tache à la fois
        #retrouver toutes les taches de la machine et stocker les startdates (et enddates) comme tuples
        #les ordonner par startdate
        #verifier que startdate+duration<nextstartdate
    
    
    return

In [ ]:
def display_detail():
    
    return

In [ ]:
def chemin_critique():
    #Calculer le chemin critique et retourner la liste de tâches qui le compose
    
    return